In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import time
import geopandas as gpd
from shapely.geometry import (LinearRing, LineString, MultiLineString,
                              MultiPoint, MultiPolygon, Point, Polygon)
from multiprocessing import Pool
import os
import re
import matplotlib.pyplot as plt

import mytools
# import importlib
# importlib.reload(mytools)

# 一、计算覆盖空洞生成图层

# 1、-----------------现网覆盖区域计算---------------------------

### 20200707 导入计算覆盖半径的数据

In [ ]:
gongcan_loc_juli = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/1-修正半径-2覆盖模型和站高输出数据.csv',encoding='gbk'))

In [ ]:
gongcan_loc_juli.head(1)

In [ ]:
gongcan_loc_juli.columns

In [ ]:
#做扇区
print('做扇区')
gongcan_s = mytools.gisn.add_sectors_df(gongcan_loc_juli,coords=['RRU经度', 'RRU纬度', 'd_height', '方位角', 'distance_use_3'], has_z=False, sec_col='geometry', shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
gongcan_s.to_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/2-修正后覆盖半径的扇区图层.data')

In [ ]:
gongcan_s=pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/2-修正后覆盖半径的扇区图层.data')

In [ ]:
gongcan_s.crs

In [ ]:
gongcan_s.plot(figsize=(11,11))

# 2、-----------------求原始覆盖空洞---------------------------

### 1-按照区县合并覆盖区域

In [ ]:
gongcan_s['归属区县'] = gongcan_s['归属区县'].fillna('wu')

In [ ]:
gongcan_s.columns

In [ ]:
gongcan_s = gongcan_s[[ '小区CGI(*)', '小区中文名', '所属地区', 'RRU经度', 'RRU纬度', '天线挂高',
       '方位角', '工作频段', '归属区县', '厂家名称', '区域类型', 'distance', '覆盖模型距离',
       '30米覆盖模型距离', '使用覆盖半径', 'distance_use', 'distance_yb', 'h_use', '高度建筑比',
       'distance_use_3', 'geometry']]

In [ ]:
gongcan_city = gongcan_s.dissolve(by = '归属区县')

In [ ]:
gongcan_city.to_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/2-修正后覆盖半径的扇区图层_按照区县合并.data')

In [ ]:
gongcan_city = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/2-修正后覆盖半径的扇区图层_按照区县合并.data')

### 2-求出覆盖空洞原始

In [ ]:
tu = mytools.gisn.maps('地市区县')

In [ ]:
city_tu_diff = gpd.overlay(tu,gongcan_city,how='difference')

In [ ]:
city_tu_diff.to_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/2-覆盖空洞1-原始.data')

In [ ]:
city_tu_diff = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/2-覆盖空洞1-原始.data')

In [ ]:
tu_mianji.plot(figsize=(11,11),color='gray')

In [ ]:
city_tu_diff.plot(figsize=(11,11),color='coral')

# 3、-------------剔除水系-------------

### 1-图中去湖泊

In [ ]:
shuixi2_hb = gpd.read_file('G:/1-规划/图层/HB_场景/湖泊、水库.tab',encoding='gbk')

In [ ]:
shuixi2_hb['fenlei']='湖泊'
shuixi2_hb = shuixi2_hb[[ 'geometry', 'fenlei']]
kongd = city_tu_diff.copy()
kongdong_new = gpd.overlay(kongd,shuixi2_hb,how='difference')

In [ ]:
kongdong_new.to_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/2-覆盖空洞1-2去除湖泊.data')

In [ ]:
kongdong_new = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/2-覆盖空洞1-2去除湖泊.data')

In [ ]:
shuixi2_hb.plot(figsize=(11,11),color='royalblue')

In [ ]:
kongdong_new.head(1)

### 2-图中去面状水系

In [ ]:
shuixi1 = gpd.read_file('G:/1-规划/图层/HB_场景/面状水系.tab',encoding='gbk')

In [ ]:
shuixi1['fenlei']='面状'

In [ ]:
shuixi1['type'] =  shuixi1.type
shuixi1_use = shuixi1.loc[shuixi1['type']=='Polygon'].append(shuixi1.loc[shuixi1['type']=='MultiPolygon'].explode())
shuixi1_use_area = mytools.gisn.add_area(shuixi1_use)
shuixi1_use_area_use = shuixi1_use_area.loc[shuixi1_use_area['面积']>100]
shuixi1_use_area_use = shuixi1_use_area_use.reset_index(drop=True)
shuixi1_use_area_use['geometry'] = shuixi1_use_area_use.buffer(0.00000001)

In [ ]:
shuixi1_use_area_use.to_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/水系图层_去水系准备.data')

In [ ]:
shuixi1_use_area_use = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/水系图层_去水系准备.data')

In [ ]:
def myYield(df):
    y = 1000
    x = 0
    while x<=236973:
        shuixi1_use_t = df.iloc[x:y,]
        yield shuixi1_use_t
        print('完成{cishu}。{cishu2}。。'.format(cishu=x,cishu2 = y))
        x = y 
        y = y + 1000

In [ ]:
tu_new_t = kongdong_new.copy()

In [ ]:
shuixi1_use_area_use.shape

In [ ]:
for data_t in myYield(shuixi1_use_area_use):
    data_t = data_t.dissolve(by='fenlei')
    tu_new_t = gpd.overlay(tu_new_t,data_t,how='difference')

In [ ]:
tu_new_t.to_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/水系图层_-2去除湖泊水系完成.data')

In [ ]:
tu_new_t.to_file('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/水系图层_-2去除湖泊水系完成.shp',encoding='utf-8')

In [ ]:
tu_new_t = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/2-计算空洞/水系图层_-2去除湖泊水系完成.data')

In [ ]:
tu_new_t.plot(figsize=(11,11),color='coral')

# 4、-------------空洞有效数据整理-------------

### 1-拆分成多个空洞

In [ ]:
tu_new_t['type'] =  tu_new_t.type

In [ ]:
tu_new_t.groupby('type').count()

In [ ]:
kongd_use = tu_new_t.loc[tu_new_t['type']=='Polygon'].append(tu_new_t.loc[tu_new_t['type']=='MultiPolygon'].explode())

In [ ]:
kongd_mianji = mytools.gisn.add_area(kongd_use)

In [ ]:
kongd_mianji.head(1)

In [ ]:
kongd_mianji.shape

In [ ]:
kongd_mianji.drop(columns='geometry').to_csv('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/3-拆分后的空洞/覆盖空22639个.csv',index=False,encoding='gbk')

In [ ]:
# kongd_mianji.to_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞1去除水系的原始空洞_修正半径后.shp',encoding='utf-8')

In [ ]:
kongd_mianji.head(1)

In [ ]:
kongd_mianji.plot(figsize=(11,11))

# 有效空洞匹配场景

In [ ]:
# kongd_valid  = gpd.read_file('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/覆盖空洞3去除冗余有效空洞7143个_修正半径后.shp',encoding='utf-8')

In [ ]:
# kongd_valid.shape

In [ ]:
kongd_mianji.crs

In [ ]:
kongd_mianji[['lon','lat']] = kongd_mianji.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

In [ ]:
result = mytools.gisn.points_coverage_merge(kongd_mianji,'lon','lat')

In [ ]:
result.shape

In [ ]:
#添加id编号
result_id = result.reset_index(drop=True)
result['id'] = [ 'kd_{}'.format(str(name)) for name in result_id.index+1]

In [ ]:
result['制式'] = 'FDD1800'
result.loc[result['区域类型']=='农村','制式']='FDD900'

In [ ]:
result = mytools.gisn.changjing_distance(result,'区域类型','制式')

In [ ]:
import numpy as np

In [ ]:
result['标准面积'] = np.pi* result['distance']* result['distance']

result['空洞面积比例'] = result['面积']/result['标准面积']

In [ ]:
def fuck_data(x):
    if x['空洞面积比例']>=1:
        return '大'
    elif x['空洞面积比例']>=1/3:
        return '中'
    elif x['空洞面积比例']>=1/6:
        return '小'
    else:
        return '微'

In [ ]:
result['空洞级别'] = result.apply(fuck_data,axis=1)

In [ ]:
result.head(1)

In [ ]:
result.to_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/3-拆分后的空洞/覆盖空22639个添加场景区分大中小微.data')

In [ ]:
result.reindex(columns=['id','地市','区县','面积','lon','lat', '区域类型','制式','distance','标准面积','空洞面积比例','空洞级别']).to_excel(
    'G:/1-规划/评估/覆盖空洞/20200708第三版本/2-程序输出/3-拆分后的空洞/覆盖空22639个添加场景区分大中小微.xlsx',index=False)